In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
from sklearn.utils import shuffle

In [2]:
def ORB(img) :
    orb = cv2.ORB_create(nfeatures=500,scoreType = cv2.ORB_FAST_SCORE)
    # find the keypoints with ORB
    kp = orb.detect(img,None)
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    return des

In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
kmeans = KMeans(init="k-means++", n_clusters=1024,n_init=10,max_iter=300,random_state=42)

In [78]:

views = ['A2c','A4c','PLAX','PSAX']
X = []
Y = []
feature_vector = []

for view in views :
    path = r"C:/Users/hp/Desktop/python_projects_n_codes/Echocodes/preprocessing/echo_images/*"+view+"*.jpg"
    for file in glob.glob(path):  
            a = cv2.imread(file,0)   #now, we can read each file since we have the full path
            a  = a[100:450,225:575]
            a = cv2.resize(a,(200,200))
            Y.append(view)
            descriptor = ORB(a)
            # mean of each descriptor ----> 500 keypoint's mean
            #keyptmean = np.mean(descriptor, axis = 1)
            #keyptmean = np.reshape(keyptmean,newshape=(keyptmean.shape[0],1))
            #cluster these 500 means into 32 clusters
            scaled_keyptmean = scaler.fit_transform(descriptor)
            kmeans.fit(descriptor)
            cluster_map = pd.DataFrame(descriptor)
            cluster_map['cluster'] = kmeans.labels_
            img_feature = []
            cluster_center = []
            for i in range(kmeans.n_clusters):
                feature_vector = np.ravel((cluster_map[cluster_map.cluster == i].drop('cluster',axis = 1))).mean() # just 1 element
                #img_feature.append(cluster_mean)    #64 feature/elemnts  64
            #feature_vector.append(img_feature)       # all images 1237,64
            #kmeans.cluster_centers_[i]

In [69]:
#print("feature extracted from 1 image's size :", len(img_feature))
#print("sample cluster mean = ",img_feature)
print("feature vector total size :", len(feature_vector))
#print("feature vector = ",feature_vector)
print("feature vector of a single image : ",len(feature_vector[0]),"  ",feature_vector[0])

feature extracted from 1 image's size : 64
sample cluster mean =  [-1.0028555503132635, 0.7614836673441457, -0.13546582092619439, 1.8218321436025378, -1.8701694124975523, 2.2477506200110926, 0.2844798375180211, 1.2051531725232192, -0.5651776574737638, 3.9317749201252163, -1.2288639473957512, 3.0077123902612697, 0.558592639175503, -0.38045050210410214, -1.4941715555184292, 1.5642629901634495, 0.09509460832311055, -0.8016557285399313, 1.0507178945436584, 2.4931051538025546, 1.3828139301805864, -0.26803617106447514, 0.8365637478955914, -1.350782542263679, 0.4722759936656063, -1.6265408575693425, -0.6860341115027675, -0.0009532703967380973, -1.9889964188521834, -0.8754745168451548, -1.753333920215904, 0.699680347279251, -0.21057435167868263, -1.0706428959974537, 0.16104217433987866, 1.105816450562845, 2.8499844756810444, 1.8996339600537264, -0.45327353337283416, -2.11446180545009, 0.622276833642659, -0.749478059827786, 0.9830595231041948, 1.6383473136784037, -0.06511416378803035, -1.155985

In [70]:
col_name = ["feature" + str(i) for i in range(1,kmeans.n_clusters + 1)]
row_name = ["image" + str(i) for i in range(1,len(Y)+1)]

In [71]:
df = pd.DataFrame(feature_vector,index=row_name,columns=col_name)
df["path"] = path
df["views"] = Y
df = shuffle(df)
df.to_csv("C:/Users/hp/Desktop/python_projects_n_codes/Echocodes/view_classification/viewfeature_orb_v2.csv")

In [72]:
df

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature57,feature58,feature59,feature60,feature61,feature62,feature63,feature64,path,views
image362,-0.752666,0.806322,1.924834,0.020232,-1.389212,-2.524255,-0.368406,1.228022,0.386780,2.595098,...,-1.938013,2.047446,0.539967,0.740855,1.029980,-0.787815,0.187565,0.344225,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,A4c
image1088,0.236515,-1.568856,0.956973,-0.756123,1.909625,-0.190091,-2.208697,3.200575,0.584336,1.384607,...,-0.706721,1.588766,-1.039921,-1.473886,-1.334650,1.453018,-0.277907,-2.051219,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,PSAX
image200,-0.409679,0.952881,-1.456863,1.602179,-0.829917,0.304677,2.865129,-1.972126,-0.072537,2.055588,...,-1.252296,-0.768642,0.718014,-0.358994,1.437606,1.850705,-1.021276,0.355553,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,A2c
image1218,0.070564,-1.301457,1.322313,-0.395346,-2.264209,3.029169,0.655313,-0.922991,1.944375,0.236196,...,-2.170655,-0.728529,-0.232160,0.131027,-0.124715,-1.757218,1.172790,0.279121,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,PSAX
image912,0.176240,-1.768845,1.758673,-0.565331,1.030968,-1.004505,0.572870,-0.128714,-2.808886,2.219624,...,-1.849766,1.639431,-1.067919,0.989729,0.142394,-2.432293,1.819947,-1.456573,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,PSAX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
image563,-0.486725,0.773325,-1.550191,2.214794,0.263202,-1.049313,1.267426,-0.150637,-2.947466,3.090405,...,-0.907079,0.322132,-0.787056,1.960227,-0.327345,-0.594280,-1.337481,-0.185490,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,PLAX
image170,0.160373,-1.413706,1.312371,-0.386018,0.722430,3.090228,-2.680741,-0.876829,2.188806,-1.961472,...,-0.035558,-1.339973,-1.686841,1.226106,-2.238904,2.273810,1.095423,0.893934,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,A2c
image367,-0.685271,1.143478,0.150377,-1.323883,2.446617,-1.803719,1.783896,0.579528,-0.195505,-0.919878,...,1.067329,-1.167124,-1.721675,1.429913,0.629311,-1.529709,-0.148737,0.474385,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,A4c
image596,0.471020,-1.172063,1.602360,-0.382083,-4.076419,2.549047,0.047562,-1.782771,0.810947,-0.770939,...,0.008699,2.640116,1.918818,-0.729882,1.524186,0.771591,-0.810026,-1.578390,C:/Users/hp/Desktop/python_projects_n_codes/Ec...,PLAX


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)